This file contains the code related the data manipulation that will be used in the Code.ipynb file in this repository.

In [1]:
#Import packages
import numpy as np
import pandas as pd
import scipy
import math
import matplotlib.pyplot as plt
import geopandas as gpd
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

/Users/jacobeppley/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/jacobeppley/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
#Load in data
pa_data = pd.read_csv("PA_EMS_Data.csv")
block_shape = gpd.read_file("allegheny_county_census_block_groups_2016.zip")

In [3]:
#Filtering to just overdoses
pa_od = pa_data.drop(pa_data[pa_data['description_short'] != 'OVERDOSE'].index)
pa_od.reset_index(inplace=True)
del pa_od["index"]
del pa_od["_id"]

In [4]:
#Grouping for analysis
group_pa_od = pa_od.groupby(['geoid','call_year',"call_quarter"]).size().reset_index(name='counts')

In [15]:
group_pa_od['geoid'] = group_pa_od.geoid.astype(int)
group_pa_od['geoid'] = group_pa_od.geoid.astype(str)

In [16]:
#Adding the geographies
coord_list = []
for i in range(len(group_pa_od["geoid"])):
    for j in range(len(block_shape["GEOID"])):
        if group_pa_od["geoid"][i] == block_shape["GEOID"][j]:
            coord_list.append(block_shape["geometry"][j])

In [17]:
group_pa_od["geometry"] = coord_list

In [18]:
#Fiddling around to create testing and training data
train_group_pa_od = group_pa_od[group_pa_od.call_year != 2024]
test_group_pa_od = group_pa_od[group_pa_od.call_year == 2024]

In [19]:
#More fiddling
condition1 = test_group_pa_od['call_quarter'] == "Q1"
rows1 = test_group_pa_od.loc[condition1, :]
condition2 = test_group_pa_od['call_quarter'] == "Q2"
rows2 = test_group_pa_od.loc[condition2, :]
combined_train = [train_group_pa_od,rows1,rows2]
train_data = pd.concat(combined_train)

In [20]:
#Test data creation
test_data = test_group_pa_od.drop(test_group_pa_od[test_group_pa_od['call_quarter'] != 'Q3'].index)

In [21]:
train_data.reset_index(inplace=True)
test_data.reset_index(inplace=True)

In [22]:
del train_data["index"]
del test_data["index"]

In [23]:
#Fiddling to get a good format
train_data['call_year'] = train_data.call_year.astype(str)
train_data["year_quarter"] = train_data["call_year"] + " " + train_data["call_quarter"]
test_data['call_year'] = test_data.call_year.astype(str)
test_data["year_quarter"] = test_data["call_year"] + " " + test_data["call_quarter"]

In [24]:
train_data

,geoid,call_year,call_quarter,counts,geometry,year_quarter
0,420030103001,2015,Q1,2,"POLYGON ((-79.9965753170118 40.4329079681144, ...",2015 Q1
1,420030103001,2015,Q2,2,"POLYGON ((-79.9965753170118 40.4329079681144, ...",2015 Q2
2,420030103001,2015,Q3,5,"POLYGON ((-79.9965753170118 40.4329079681144, ...",2015 Q3
3,420030103001,2015,Q4,9,"POLYGON ((-79.9965753170118 40.4329079681144, ...",2015 Q4
4,420030103001,2016,Q1,7,"POLYGON ((-79.9965753170118 40.4329079681144, ...",2016 Q1
...,...,...,...,...,...,...
18968,420039806001,2024,Q2,11,"POLYGON ((-80.0373403311079 40.4611129710906, ...",2024 Q2
18969,420039807001,2024,Q2,1,"POLYGON ((-80.0285033269786 40.4439509681772, ...",2024 Q2
18970,420039808001,2024,Q2,1,"POLYGON ((-79.9734093100364 40.4342729693668, ...",2024 Q2
18971,420039812001,2024,Q2,7,"POLYGON ((-80.0180713241808 40.4488909698489, ...",2024 Q2


In [ ]:
#Run here to save CSV files
#Done so we don't have to run the 5+ minutes of code every time for the geography assignment
pd.train_data.to_csv("train_data.csv")
pd.test_data.to_csv("test_data.csv")